# Learning from Demonstration in Robotics

This notebook demonstrates the concept of learning from demonstration for robotic control, as discussed in Chapter 8 of the Physical AI & Humanoid Robotics book.

In [ ]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")

## 1. Learning from Demonstration Class

First, let's implement a Learning from Demonstration class that can learn robotic control policies from demonstration data.

In [ ]:
class LearningFromDemonstration:
    """A class to implement Learning from Demonstration for robotic control."""
    
    def __init__(self, hidden_layers=(100, 50), random_state=42):
        """
        Initialize the Learning From Demonstration system.
        
        Args:
            hidden_layers: Tuple defining the neural network architecture
            random_state: Random seed for reproducibility
        """
        self.model = MLPRegressor(
            hidden_layer_sizes=hidden_layers, 
            max_iter=1000, 
            random_state=random_state
        )
        self.is_trained = False
        
    def train(self, demonstrations):
        """
        Train the model on demonstrations.
        
        Args:
            demonstrations: List of (state, action) tuples
            
        Returns:
            Trained model
        """
        if len(demonstrations) == 0:
            raise ValueError("Demonstrations list cannot be empty")
        
        # Extract states and actions from demonstrations
        states = np.array([demo[0] for demo in demonstrations])
        actions = np.array([demo[1] for demo in demonstrations])
        
        # Validate dimensions
        if states.shape[0] != actions.shape[0]:
            raise ValueError("Number of states and actions must match")
        
        # Train the model
        self.model.fit(states, actions)
        self.is_trained = True
        return self.model
    
    def predict_action(self, state):
        """
        Predict action for given state.
        
        Args:
            state: Current state vector
            
        Returns:
            Predicted action
        """
        if not self.is_trained:
            raise ValueError("Model must be trained first")
        
        state = np.array(state).reshape(1, -1)  # Ensure proper shape
        return self.model.predict(state)[0]
    
    def evaluate(self, test_states, test_actions):
        """
        Evaluate the model performance on test data.
        
        Args:
            test_states: Array of test states
            test_actions: Array of test actions
            
        Returns:
            Mean squared error
        """
        if not self.is_trained:
            raise ValueError("Model must be trained first")
        
        predictions = self.model.predict(test_states)
        mse = mean_squared_error(test_actions, predictions)
        return mse

## 2. Generate Simulated Demonstrations

Now let's generate simulated demonstrations for a simple robotic task. We'll simulate a reaching task where the robot learns to move its end-effector toward a target.

In [ ]:
def generate_simulated_demonstrations():
    """
    Generate simulated demonstrations for a robotic arm.
    This simulates a simple reaching task where the robot learns
    to move its end-effector toward a target.
    
    Returns:
        demonstrations: List of (state, action) tuples
    """
    demonstrations = []
    np.random.seed(42)  # For reproducibility
    
    for _ in range(200):  # Generate 200 demonstration samples
        # State variables: current joint angles (3 joints) and target position (3D)
        current_angles = np.random.uniform(-np.pi/2, np.pi/2, 3)
        target_pos = np.random.uniform(-1.0, 1.0, 3)
        
        # State vector includes current angles and target position
        state = np.concatenate([current_angles, target_pos])
        
        # Action: desired joint velocity changes to move toward target
        # This is a simplified model where we want to reduce the distance to target
        # In a real system, this would come from a human demonstrator
        joint_velocities = np.zeros(3)
        
        # Calculate desired joint changes to move toward target
        # This is a simplified kinematic model
        for i in range(3):
            # The desired velocity is proportional to the error in the corresponding dimension
            # We also include a small amount of noise to make it more realistic
            joint_velocities[i] = (target_pos[i] - np.sin(current_angles[i])) * 0.1 + \
                                 np.random.normal(0, 0.01)
        
        action = joint_velocities
        demonstrations.append((state, action))
    
    return demonstrations

# Generate demonstrations
print("Generating simulated demonstrations...")
demonstrations = generate_simulated_demonstrations()
print(f"Generated {len(demonstrations)} demonstration samples")

## 3. Train the Learning from Demonstration Model

Now we'll train our model on the generated demonstrations.

In [ ]:
# Separate states and actions
states = np.array([demo[0] for demo in demonstrations])
actions = np.array([demo[1] for demo in demonstrations])

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    states, actions, test_size=0.2, random_state=42
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# Create and train the LfD system
print("\nTraining the Learning from Demonstration model...")
lfd = LearningFromDemonstration()
lfd.train(list(zip(X_train, y_train)))
print("Training completed!")

# Evaluate the model
train_mse = lfd.evaluate(X_train, y_train)
test_mse = lfd.evaluate(X_test, y_test)

print(f"\nModel Performance:")
print(f"Training MSE: {train_mse:.4f}")
print(f"Testing MSE: {test_mse:.4f}")

## 4. Test the Model

Let's test the trained model on a new example and visualize the results.

In [ ]:
# Demonstrate prediction on a new example
print("\nTesting prediction on a new example:")
new_state = np.concatenate([
    np.array([0.1, -0.2, 0.3]),    # Current joint angles
    np.array([0.5, 0.3, -0.2])     # Target position
])

predicted_action = lfd.predict_action(new_state)
print(f"Input state (angles + target): {new_state}")
print(f"Predicted action (joint velocities): {predicted_action}")

# Make predictions on test data for visualization
print("\nGenerating predictions for visualization...")
predictions = lfd.model.predict(X_test)

## 5. Visualization

Let's visualize the performance of our Learning from Demonstration model.

In [ ]:
# Create plots to visualize the results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Comparison of predicted vs actual actions for Joint 1
axes[0].scatter(y_test[:, 0], predictions[:, 0], alpha=0.6, color='blue')
axes[0].plot([y_test[:, 0].min(), y_test[:, 0].max()], 
             [y_test[:, 0].min(), y_test[:, 0].max()], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Action (Joint 1)')
axes[0].set_ylabel('Predicted Action (Joint 1)')
axes[0].set_title('Joint 1 Predictions vs Actual')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Comparison for Joint 2
axes[1].scatter(y_test[:, 1], predictions[:, 1], alpha=0.6, color='green')
axes[1].plot([y_test[:, 1].min(), y_test[:, 1].max()], 
             [y_test[:, 1].min(), y_test[:, 1].max()], 'r--', lw=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Action (Joint 2)')
axes[1].set_ylabel('Predicted Action (Joint 2)')
axes[1].set_title('Joint 2 Predictions vs Actual')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Comparison for Joint 3
axes[2].scatter(y_test[:, 2], predictions[:, 2], alpha=0.6, color='orange')
axes[2].plot([y_test[:, 2].min(), y_test[:, 2].max()], 
             [y_test[:, 2].min(), y_test[:, 2].max()], 'r--', lw=2, label='Perfect Prediction')
axes[2].set_xlabel('Actual Action (Joint 3)')
axes[2].set_ylabel('Predicted Action (Joint 3)')
axes[2].set_title('Joint 3 Predictions vs Actual')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate and display correlation coefficients
correlations = []
for i in range(3):
    corr = np.corrcoef(y_test[:, i], predictions[:, i])[0, 1]
    correlations.append(corr)
    print(f"Correlation for Joint {i+1}: {corr:.3f}")

## 6. Advanced Example: Learning from Human Demonstrations

Let's create a more advanced example with a simulated human demonstration for a pick-and-place task.

In [ ]:
def simulate_human_demonstration_for_pick_place():
    """
    Simulate human demonstrations for a pick-and-place task.
    This simulates a human operator demonstrating how to perform
    a pick-and-place task with a robotic arm.
    
    Returns:
        demonstrations: List of (state, action) tuples
    """
    demonstrations = []
    np.random.seed(123)
    
    # Define a pick-and-place task
    for _ in range(300):
        # State includes: current end-effector position, object position, target position
        current_pos = np.random.uniform(-0.5, 0.5, 3)  # Current EE position
        obj_pos = np.random.uniform(-0.8, -0.2, 3)     # Object position
        target_pos = np.random.uniform(0.2, 0.8, 3)   # Target position
        
        state = np.concatenate([current_pos, obj_pos, target_pos])
        
        # Simulate human-like behavior: approach object, grasp, move to target, release
        # This is a simplified kinematic model of how a human would approach the task
        action = np.zeros(6)  # 3 for position, 3 for orientation
        
        # Calculate desired movement direction
        if np.linalg.norm(current_pos - obj_pos) > 0.1:
            # Approach object
            direction_to_obj = obj_pos - current_pos
            action[:3] = direction_to_obj * 0.1
        elif np.linalg.norm(current_pos - target_pos) > 0.1:
            # Move to target
            direction_to_target = target_pos - current_pos
            action[:3] = direction_to_target * 0.1
        else:
            # At target, minimal movement
            action[:3] = np.random.normal(0, 0.01, 3)
        
        # Add some random orientation adjustments
        action[3:6] = np.random.uniform(-0.05, 0.05, 3)
        
        demonstrations.append((state, action))
    
    return demonstrations

# Generate more complex demonstrations
complex_demonstrations = simulate_human_demonstration_for_pick_place()
print(f"Generated {len(complex_demonstrations)} complex pick-and-place demonstrations")

# Prepare data
complex_states = np.array([demo[0] for demo in complex_demonstrations])
complex_actions = np.array([demo[1] for demo in complex_demonstrations])

# Split data
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    complex_states, complex_actions, test_size=0.2, random_state=42
)

# Train a new model on the complex task
print("\nTraining model on complex pick-and-place task...")
complex_lfd = LearningFromDemonstration(hidden_layers=(128, 64, 32))
complex_lfd.train(list(zip(X_train_c, y_train_c)))

# Evaluate
train_mse_c = complex_lfd.evaluate(X_train_c, y_train_c)
test_mse_c = complex_lfd.evaluate(X_test_c, y_test_c)

print(f"Complex Task - Training MSE: {train_mse_c:.4f}")
print(f"Complex Task - Testing MSE: {test_mse_c:.4f}")

## 7. Summary and Conclusions

In this notebook, we implemented and demonstrated Learning from Demonstration (LfD) for robotic control:

1. We created a `LearningFromDemonstration` class that uses neural networks to learn control policies
2. We generated simulated demonstrations for a reaching task
3. We trained the model and evaluated its performance
4. We visualized the results showing the correlation between predicted and actual actions
5. We extended the approach to a more complex pick-and-place task

LfD is a powerful approach for teaching robots complex behaviors by demonstration, avoiding the need to manually program complex control policies. The approach works well when good demonstrations are available, but quality of demonstrations directly affects the learned policy.